In [ ]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
from torch import optim
from torch.utils.data import Dataset, DataLoader
from torchvision.datasets import ImageFolder
from torchvision.transforms import Compose, Normalize, RandomHorizontalFlip, Resize, ToTensor
from transformers import get_cosine_schedule_with_warmup
from tqdm.auto import tqdm
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

### Patch Embedding

In [ ]:
class PatchEmbed(nn.Module):
    def __init__(self, img_size=224, patch_size=16, in_chans=3, embed_dim=786):
        super(PatchEmbed, self).__init__()
        self.img_size = img_size
        self.patch_size = patch_size
        self.in_chans = in_chans
        self.embed_dim = embed_dim

        self.n_patches = (img_size // patch_size) ** 2

        self.proj = nn.Conv2d(in_channels=in_chans,
                              out_channels=embed_dim,
                              kernel_size=patch_size,
                              stride=patch_size)

    def forward(self, x):
        x = self.proj(x)
        x = x.flatten(2) 
        x = x.transpose(1,2) 
        return x

### Attention

In [ ]:

class Head(nn.Module):
    def __init__(self, embed_dim=768, head_dim=768, attn_p=0):
        super(Head, self).__init__()
        self.query = nn.Linear(embed_dim, head_dim)
        self.key = nn.Linear(embed_dim, head_dim)
        self.value = nn.Linear(embed_dim, head_dim)
        self.attn_dropout = nn.Dropout(attn_p)

    def forward(self, x):
        batch_size, n_patch, embed_dim = x.shape
        q = self.query(x) 
        k = self.key(x)
        v = self.value(x) 

        sam = (q @ k.transpose(-2,-1)) * embed_dim**-0.5 
        attn = sam.softmax(dim=-1)
        attn = self.attn_dropout(attn)
        weighted_average = attn @ v 
        return weighted_average
    
class MultiHeadedAttention(nn.Module):
    def __init__(self, embed_dim=768, num_heads=12, attn_p=0, proj_p=0):
        super(MultiHeadedAttention, self).__init__()
        self.head_size = embed_dim // num_heads
        self.heads = nn.ModuleList([Head(embed_dim=embed_dim, head_dim=self.head_size, attn_p=attn_p) for _ in range(num_heads)])
        self.proj = nn.Linear(embed_dim, embed_dim)
        self.proj_drop = nn.Dropout(proj_p)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1) 
        out = self.proj_drop(self.proj(out)) 
        return out

### Efficient Attention

In [ ]:
class EfficientAttention(nn.Module):
    def __init__(self, embed_dim, num_heads, attn_p, proj_p):
        super(EfficientAttention, self).__init__()
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.head_size = int(self.embed_dim / num_heads)

        self.qkv = nn.Linear(embed_dim, embed_dim*3)
        self.attn_dropout = nn.Dropout(attn_p)
        self.proj = nn.Linear(embed_dim, embed_dim)
        self.proj_drop = nn.Dropout(proj_p)

    def forward(self, x):
        batch, patches, embed_dim = x.shape 
        qkv = self.qkv(x)
        qkv = qkv.reshape(batch, patches, 3, self.num_heads, self.head_size) 
        qkv = qkv.permute(2, 0, 3, 1, 4)  
        q, k, v = qkv[0], qkv[1], qkv[2]
        
        sam = (q @ k.transpose(-2,-1)) * self.head_size**-0.5 
        attn = sam.softmax(dim=-1)
        attn = self.attn_dropout(attn)
        weighted_average = attn @ v 
        weighted_average = weighted_average.transpose(1,2) 
        weighted_average = weighted_average.flatten(2) 
        out = self.proj_drop(self.proj(weighted_average))
        return out

In [ ]:
class MLP(nn.Module):
    def __init__(self, in_features, hidden_features, out_features, mlp_p=0):
        
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(in_features, hidden_features)
        self.act = nn.GELU()
        self.fc2 = nn.Linear(hidden_features, out_features)
        self.drop = nn.Dropout(mlp_p)

    def forward(self, x):
        x = self.act(self.fc1(x)) 
        x = self.drop(x)
        x = self.fc2(x) 
        x = self.drop(x)
        return x

### Transformer Block

In [ ]:
class TransformerBlock(nn.Module):
    def __init__(self,embed_dim= 768, num_heads= 12, mlp_ratio = 4.0, mlp_p= 0, attn_p = 0,proj_p = 0,efficient = True):
        super(TransformerBlock,self).__init__()
        self.norm1 = nn.LayerNorm(embed_dim,eps = 1e-6)

        if efficient:
            self.attn = EfficientAttention(
                embed_dim= embed_dim,
                num_heads=num_heads,
                attn_p=attn_p,
                proj_p=proj_p,
            )
        else:
            self.attn = MultiHeadedAttention(
                embed_dim= embed_dim,
                num_heads=num_heads,
                attn_p=attn_p,
                proj_p=proj_p
            )

        self.norm2 = nn.LayerNorm(embed_dim,eps = 1e-6)
        hidden_features = int(embed_dim*mlp_ratio)
        self.mlp = MLP(
            in_features= embed_dim,
            hidden_features= hidden_features,
            out_features=embed_dim,
            mlp_p=mlp_p
        )
    def forward(self,x):
        x = x + self.attn(self.norm1(x))
        x = x + self.mlp(self.norm2(x))
        return x

In [ ]:
class VisionTransformer(nn.Module):
    def __init__(self,img_size = 224 , patch_size =16,in_chans = 3, n_classes = 2,
                 embed_dim = 768, depth = 12, num_heads = 12, mlp_ratio = 4, attn_p = 0.2,
                 mlp_p = 0.2, proj_p = 0.2, pos_drop = 0.2, efficient = True):
        super(VisionTransformer,self).__init__()
        
        self.patch_embed = PatchEmbed(
            img_size=img_size,
            patch_size=patch_size,
            in_chans=in_chans,
            embed_dim=embed_dim
        )
        self.cls_token = nn.Parameter(torch.zeros(1,1,embed_dim))
        self.pos_embed = nn.Parameter((torch.zeros(1,1+self.patch_embed.n_patches,embed_dim)))
        self.pos_drop = nn.Dropout(pos_drop)
        
        self.blocks = nn.ModuleList(
            [
                TransformerBlock(
                    embed_dim=embed_dim,
                    num_heads=num_heads,
                    mlp_ratio=mlp_ratio,
                    mlp_p=mlp_p,
                    attn_p=attn_p,
                    proj_p=proj_p,
                    efficient= efficient
                )
                for _ in range(depth)
            ]
        )

        self.norm = nn.LayerNorm(embed_dim,eps = 1e-6)
        self.head = nn.Linear(embed_dim,n_classes)
    
    def forward(self,x):
        batch_size = x.shape[0]
        x = self.patch_embed(x)
        cls_token = self.cls_token.expand(batch_size,-1,-1)
        x = torch.cat((cls_token,x),dim = 1)
        x = x+ self.pos_embed
        x = self.pos_drop(x)

        for block in self.blocks:
            x = block(x)
        
        x = self.norm(x)
        cls_token_final = x[:,0]
        x = self.head(cls_token_final)
        return x,cls_token_final

###  Training

In [ ]:
def train(model,device,epochs,optimizer,scheduler,loss_fn,train_loader,val_loader,savepath = "ViT.pt"):
    log_training = {
        "epoch": [],
        "training_loss": [],
        "training_acc": [],
        "validation_loss": [],
        "validation_acc": []
    }

    best_val_loss = np.inf
    for epoch in range(1,epochs+1):
        print(f"Epoch: {epoch}")
        training_losses , training_accuracies = [],[]
        validation_losses , validation_accuracies = [], []

        for image,label in tqdm(train_loader):
            image, label = image.to(device) , label.to(device)
            optimizer.zero_grad()
            out, _ = model.forward(image)

            loss = loss_fn(out,label)
            training_losses.append(loss.item())

            predictions = torch.argmax(out,axis=1)
            accuracy = (predictions==label).sum()/len(predictions)
            training_accuracies.append(accuracy.item())

            loss.backward()
            optimizer.step()
            scheduler.step()

        for image,label in tqdm(val_loader):
            image,label = image.to(device), label.to(device)
            with torch.no_grad():
                out, _ = model.forward(image)

                loss = loss_fn(out,label)
                validation_losses.append(loss.item())

                predictions = torch.argmax(out,axis = 1)
                accuracy = (predictions==label).sum() / len(predictions)
                validation_accuracies.append(accuracy.item())

        training_loss_mean , training_acc_mean = np.mean(training_losses) , np.mean(training_accuracies)
        validation_loss_mean,  validation_acc_mean = np.mean(validation_losses), np.mean(validation_accuracies)

        if validation_loss_mean<best_val_loss:
            print("----Saving Model-------")
            torch.save(model.state_dict(),savepath)
            best_val_loss = validation_loss_mean
        
        log_training["epoch"].append(epoch)
        log_training["training_loss"].append(training_loss_mean)
        log_training["training_acc"].append(training_acc_mean)
        log_training["validation_loss"].append(validation_loss_mean)
        log_training["validation_acc"].appenf(validation_acc_mean)

        print(f"Training Loss: {training_loss_mean}")
        print(f"Training Acc: {training_acc_mean}")
        print(f"Validation Loss: {validation_loss_mean}")
        print(f"Validation Acc: {validation_acc_mean}")
    return log_training, model

In [ ]:
ViT = VisionTransformer(
    embed_dim=384,
    depth = 6,
    num_heads= 6,
    efficient= True
)

params = sum([np.prod(p.size()) for p in ViT.parameters()])
print(f"Total Number of parameters: {params}")

path_to_data = "./dogsvscats"
dataset = ImageFolder(path_to_data)

normalizer = Normalize(mean = [0.485, 0.456,0.406],std = [0.229,0.224,0.225])
train_transforms = Compose([
    Resize((224,224)),
    RandomHorizontalFlip(),
    ToTensor(),
    normalizer
])

val_transforms = Compose([
    Resize((224,224)),
    ToTensor(),
    normalizer
])

train_samples , test_samples = int(0.9 * len(dataset)) , len(dataset) - int(0.9 * len(dataset))
train_dataset , val_dataset = torch.utils.data.random_split(dataset, lengths = [train_samples,test_samples])

train_dataset.dataset.transforms = train_transforms
val_dataset.dataset.transforms = val_transforms

device = "cuda" if torch.cuda.is_available() else "cpu"

optimizer = optim.AdamW(params = ViT.parameters(),lr = 1e-3)

loss_fn = nn.CrossEntropyLoss()

batch_size = 128
train_loader  = DataLoader(train_dataset,batch_size = batch_size, shuffle = True)
val_loader = DataLoader(val_dataset, batch_size = batch_size, shuffle = False)

epochs = 10
scheduler = get_cosine_schedule_with_warmup(optimizer,
                                            num_warmup_steps = 250,
                                            num_training_steps = epochs * (len(train_loader))
                                            )

loss , model = train(model= ViT.to(device),
                     device = device,
                     epochs=epochs,
                     optimizer=optimizer,
                     scheduler=scheduler,
                     loss_fn=loss_fn,
                     train_loader=train_loader,
                     val_loader=val_loader
                     )